In [2]:
# source: https://www.datacamp.com/community/tutorials/discovering-hidden-topics-python
import os
import pandas as pd
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

# Load Dataset
documents_list = []
with open( os.path.join("/Users/rawassizadeh/EVERYTHING/Work/TEACHING/CS 688_WebAnalyticsMining/toGithub/Clustering/articles.txt") ,"r") as fin:
    for line in fin.readlines():
        text = line.strip()
        documents_list.append(text)
        
# Initialize regex tokenizer
tokenizer = RegexpTokenizer(r'\w+')

# Vectorize document using TF-IDF
tfidf = TfidfVectorizer(lowercase=True,
                        stop_words='english',
                        ngram_range = (1,1),
                        tokenizer = tokenizer.tokenize)

# Fit and Transform the documents
train_data = tfidf.fit_transform(documents_list)   

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_extraction/image.py:172: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):


In [3]:
# Define the number of topics or components
num_components=10

# Create SVD object
lsa = TruncatedSVD(n_components=num_components, n_iter=100, random_state=42)

# Fit SVD model on data
lsa.fit_transform(train_data)

# Get Singular values and Components 
Sigma = lsa.singular_values_ 
V_transpose = lsa.components_.T

# Print the topics with their terms
terms = tfidf.get_feature_names()

for index, component in enumerate(lsa.components_):
    zipped = zip(terms, component)
    top_terms_key=sorted(zipped, key = lambda t: t[1], reverse=True)[:5]
    top_terms_list=list(dict(top_terms_key).keys())
    print("Topic "+str(index)+": ",top_terms_list)

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Topic 0:  ['s', 'trump', 'said', 'eu', 't']
Topic 1:  ['trump', 'clinton', 'republican', 'donald', 'cruz']
Topic 2:  ['s', 'league', 'season', 'min', 'leicester']
Topic 3:  ['eu', 'league', 'min', 'season', 'brexit']
Topic 4:  ['bank', 'banks', 'banking', 'rbs', 'financial']
Topic 5:  ['health', 'nhs', 'care', 'mental', 'patients']
Topic 6:  ['min', 'ball', 'corner', 'yards', 'goal']
Topic 7:  ['facebook', 'internet', 'online', 'users', 'twitter']
Topic 8:  ['film', 'films', 'movie', 'women', 'director']
Topic 9:  ['labour', 'party', 'bank', 'corbyn', 'film']


In [4]:
! pip install stop_words
! pip install pyLDAvis

#-------------- LDA example -----------
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim


/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


/opt/anaconda3/lib/python3.8/site-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [5]:
tokenizer = RegexpTokenizer(r'\w+')

# create English stop words list
en_stop = get_stop_words('en')

# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()
    
# create sample documents
doc_a = "Brocolli is good to eat. My brother likes to eat good brocolli, but not my mother."
doc_b = "My mother spends a lot of time driving my brother around to baseball practice."
doc_c = "Some health experts suggest that driving may cause increased tension and blood pressure."
doc_d = "I often feel pressure to perform well at school, but my mother never seems to drive my brother to do better."
doc_e = "Health professionals say that brocolli is good for your health." 

# compile sample documents into a list
doc_set = [doc_a, doc_b, doc_c, doc_d, doc_e]

# list for tokenized documents in loop
texts = []

# loop through document list
for i in doc_set:
    
    # clean and tokenize document string
    raw = i.lower()
    tokens = tokenizer.tokenize(raw)

    # remove stop words from tokens
    stopped_tokens = [i for i in tokens if not i in en_stop]
    
    # stem tokens
    stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
    
    # add tokens to list
    texts.append(stemmed_tokens)

# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(texts)
    
# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts]

# generate LDA model
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=2, id2word = dictionary, passes=20)

doc_lda = ldamodel[corpus]
print(ldamodel.print_topics())



[(0, '0.086*"health" + 0.086*"brocolli" + 0.086*"good" + 0.061*"eat" + 0.037*"blood" + 0.037*"caus" + 0.037*"may" + 0.037*"suggest" + 0.037*"expert" + 0.037*"tension"'), (1, '0.068*"mother" + 0.068*"brother" + 0.068*"drive" + 0.041*"pressur" + 0.040*"around" + 0.040*"spend" + 0.040*"basebal" + 0.040*"lot" + 0.040*"time" + 0.040*"practic"')]


/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()

vis = pyLDAvis.gensim_models.prepare(ldamodel, corpus, dictionary)
vis

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/opt/anaconda3/lib/python3.8/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
0      0.071064  0.0       1        1  53.145931
1     -0.071064  0.0       2        1  46.854069, topic_info=        Term      Freq     Total Category  logprob  loglift
16    health  2.000000  2.000000  Default  30.0000  30.0000
0   brocolli  2.000000  2.000000  Default  29.0000  29.0000
3       good  2.000000  2.000000  Default  28.0000  28.0000
2        eat  1.000000  1.000000  Default  27.0000  27.0000
6     around  1.000000  1.000000  Default  26.0000  26.0000
..       ...       ...       ...      ...      ...      ...
13     blood  0.292128  1.187372   Topic2  -4.3011  -0.6442
2        eat  0.292733  1.785463   Topic2  -4.2990  -1.0500
3       good  0.292735  2.383572   Topic2  -4.2990  -1.3390
0   brocolli  0.292729  2.383572   Topic2  -4.2990  -1.3390
16    health  0.292495  2.383579   Topic2  -4.2998  -1.3398

[92 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
6         2  0.854775      around
7         2  0.854774     basebal
22        2  0.854766      better
13        1  0.842196       blood
0         1  0.839077    brocolli
1         1  0.425933     brother
1         2  0.425933     brother
14        1  0.842196        caus
8         1  0.425891       drive
8         2  0.425891       drive
2         1  0.560079         eat
15        1  0.842196      expert
23        2  0.854762        feel
3         1  0.839077        good
16        1  0.839074      health
17        1  0.842196     increas
4         1  0.842207        like
9         2  0.854774         lot
18        1  0.842196         may
5         1  0.425933      mother
5         2  0.425933      mother
24        2  0.854765       never
25        2  0.854764       often
26        2  0.854761     perform
10        2  0.854773     practic
19        1  0.565664     pressur
19        2  0.565664     pressur
30        1  0.842204  profession
31        1  0.842204         say
27        2  0.854766      school
28        2  0.854765        seem
11        2  0.854774       spend
20        1  0.842196     suggest
21        1  0.842196     tension
12        2  0.854774        time
29        2  0.854763        well, R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2])

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
